#  Setup stuff: Connectivity

In [ ]:
import os

from katana import remote


os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

my_client = remote.Client()

print(my_client)

In [ ]:
DB_NAME    = "my_db"
GRAPH_NAME = "my_graph"

print("--")

In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

#  Setup stuff: Debugging

In [ ]:
#  LIST ALL GRAPHS

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "     DB GRAPH: ", l_graph.name, "    GRAPH VERSION: ", l_graph.version)

print("--")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
my_graph.query("CALL graph.schema() RETURN *")


In [ ]:
my_client.widgets().operations()


In [ ]:
# ##################################################################

In [ ]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


#  Finally now, the Analytics ..

In [ ]:
#  https://stackoverflow.com/questions/27600608/traverse-path-cypher-query

#  The full graph

l_result = my_graph.query("""

   MATCH (n: Airport )  - [ r ] ->  (m: Airport)
   
   //  No data, there is no direct flight
   //
   // WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


In [ ]:
#  First traversal, Airport --> Airport, up to 3 in distance ..

l_result = my_graph.query("""

   MATCH (n: Airport )  - [r *..3] ->  (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code, m.airport_code
   
   """)

print(l_result[0:30])

#    m.airport_code n.airport_code
#  0            MKE            SJC
#  1            MKE            SJC

#  So Yes, answer, but no path ..


In [ ]:
#  Adding path ..

l_result = my_graph.query("""

   MATCH p= ( n: Airport )  - [r *..3] ->  (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code AS FROM, m.airport_code AS TO, p
   """)

print(l_result[0:30])

#  Currently, error ..
#
#  OperationError: query 9tWneccAey6jzpKLGGtto7v3X1iSkQDdRrSa7gcuv8fD-3qJysJrdkEo1NWz7V error: backtrace
#  (QueryClient.cpp:456): opgraph check failed (OpGraph.cpp:429): Unimplemented Features Pass
#  (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:360): backtrace
#  (OpGraph.cpp:682): backtrace (OpGraph.cpp:682): backtrace (OpGraph.cpp:682): backtrace
#  (OpGraph.cpp:682): backtrace (OpGraph.cpp:658): backtrace (UnimplementedFeaturesChecker.cpp:214):
#  accessing path variable p is not supported (UnimplementedFeaturesChecker.cpp:24): InternalError:NotImplemented


In [ ]:
#  Using COLLECT to try and get path ..

l_result = my_graph.query("""

   MATCH ( n: Airport )  - [r *..3] ->  (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   
   RETURN n.airport_code AS FROM, COLLECT(m.airport_code)
   // RETURN COLLECT( [n.airport_code, m.airport_code] )                //  This did notthing effective
   
   """)

print(l_result[0:30])

#    COLLECT(m.airport_code) FROM
#  0              [MKE, MKE]  SJC

#  Not what we want


In [ ]:
#  Another attempt at using COLLECT 

l_result = my_graph.query("""

   MATCH ( n: Airport )  - [r *..3] ->  (m: Airport)
   WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   RETURN n.airport_code AS FROM, COLLECT(r.END_ID)
   """)

print(l_result[0:30])


#  OperationError: query 6jxiUgnCFao3yf2BTXPJqQDUTkAwTL4LMzmKjF6kNmD4-4BeDpVqisXe9Ha8QW error:
#  backtrace (QueryClient.cpp:452): backtrace (OpGraph.cpp:313): backtrace (OpGraph.h:588):
#  backtrace (OpGraph.h:588): backtrace (OpGraph.h:588): backtrace (OpGraph.h:588): backtrace
#  (OpGraph.cpp:320): backtrace (RewritePattern.h:46): backtrace (MatchOrderOptimizer.cpp:102):
#  backtrace (MatchOperator.cpp:2062): returning named variable paths not yet supported
#  (MatchOperator.cpp:1894): InternalError:NotImplemented


In [ ]:
# ##################################################################

In [ ]:
#  Second traversal, Airport --> Airport, up to 3 in distance, but in other direction ..

l_result = my_graph.query("""

   MATCH (n: Airport )  - [r: FLIES_TO *..3] ->  (m: Airport)
   WHERE n.airport_code = "MKE" AND m.airport_code = "SJC"
   RETURN n.airport_code, m.airport_code
   
   """)

print(l_result[0:30])
